In [1]:
import pandas as pd
import numpy as np

In [29]:
exchange_rate = pd.read_csv("datasets/exchange_rate.txt", sep = ',', header = None)
exchange_rate.shape

(7588, 8)

In [44]:
df = exchange_rate

In [45]:
test_size = df.shape[0] - round(df.shape[0] * 90 / 100)

In [46]:
train = df[0:-test_size]
train.shape

(6829, 8)

In [47]:
test = df[-test_size:]
test.shape

(759, 8)

In [48]:
# https://www.analyticsvidhya.com/blog/2020/10/multivariate-multi-step-time-series-forecasting-using-stacked-lstm-sequence-to-sequence-autoencoder-in-tensorflow-2-0-keras/
# https://analyticsindiamag.com/how-to-do-multivariate-time-series-forecasting-using-lstm/

def split_series(series, n_past, n_future):

    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)

In [49]:
n_past = 10
n_future = 1
n_features = df.shape[1]

In [50]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [51]:
# https://www.kaggle.com/code/sasakitetsuya/multivariate-time-series-forecasting-with-lstms

from tensorflow import keras

encoder_inputs = keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

decoder_inputs = keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

decoder_l1 = keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = keras.layers.TimeDistributed(keras.layers.Dense(n_features))(decoder_l1)

model = keras.models.Model(encoder_inputs,decoder_outputs1)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10, 8)]      0           []                               
                                                                                                  
 lstm_3 (LSTM)                  [(None, 100),        43600       ['input_2[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 repeat_vector_1 (RepeatVector)  (None, 1, 100)      0           ['lstm_3[0][0]']                 
                                                                                            

In [54]:
model.compile(optimizer=keras.optimizers.Adam(), loss='mse')

In [56]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/20
214/214 [==============================] - 2s 11ms/step - loss: 4.6074e-05 - val_loss: 1.0509e-04
Epoch 2/20
214/214 [==============================] - 2s 10ms/step - loss: 4.9536e-05 - val_loss: 4.8608e-05
Epoch 3/20
214/214 [==============================] - 2s 10ms/step - loss: 4.7768e-05 - val_loss: 6.8567e-05
Epoch 4/20
214/214 [==============================] - 2s 10ms/step - loss: 5.6377e-05 - val_loss: 1.3278e-04
Epoch 5/20
214/214 [==============================] - 2s 10ms/step - loss: 4.8827e-05 - val_loss: 8.2112e-05
Epoch 6/20
214/214 [==============================] - 2s 10ms/step - loss: 4.7479e-05 - val_loss: 5.9415e-05
Epoch 7/20
214/214 [==============================] - 2s 10ms/step - loss: 4.9944e-05 - val_loss: 1.4720e-04
Epoch 8/20
214/214 [==============================] - 2s 10ms/step - loss: 4.1617e-05 - val_loss: 6.3115e-05
Epoch 9/20
214/214 [==============================] - 2s 10ms/step - loss: 4.2911e-05 - val_loss: 6.2111e-05
Epoch 10/20
214/214

In [57]:
predicted = model.predict(X_test)

24/24 [==============================] - 1s 4ms/step


In [59]:
from sklearn.metrics import mean_squared_error
for index, i in enumerate(train.columns):
    rmse = np.sqrt(mean_squared_error(y_test[:,0,index],predicted[:,0,index]))
    print('RMSE for {}: {:.3f}'.format(i, rmse))
    nrmse_mean = rmse / (y_test[:,0,index].mean())
    print('NRMSE (mean) for {}: {:.3f}'.format(i, nrmse_mean))

RMSE for 0: 0.009
NRMSE (mean) for 0: 0.012
RMSE for 1: 0.014
NRMSE (mean) for 1: 0.010
RMSE for 2: 0.005
NRMSE (mean) for 2: 0.006
RMSE for 3: 0.011
NRMSE (mean) for 3: 0.011
RMSE for 4: 0.004
NRMSE (mean) for 4: 0.026
RMSE for 5: 0.002
NRMSE (mean) for 5: 0.184
RMSE for 6: 0.009
NRMSE (mean) for 6: 0.013
RMSE for 7: 0.004
NRMSE (mean) for 7: 0.006


In [62]:
filename = './lstm_model'
model.save(filename)

INFO:tensorflow:Assets written to: ./lstm_model\assets


INFO:tensorflow:Assets written to: ./lstm_model\assets
